In [1]:
import pandas as pd
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import joblib
import math

In [4]:
# df = pd.read_csv('data/data_aposentadoria/labeled.csv')
df = pd.read_csv('data/labeled.csv')
print(len(df))

241


In [5]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq):
    x = []
    y = []
    for i in range(len(arq)):
        x.append(extract_features(arq['text'][i].split()))
        y.append(arq['labels'][i].split())
    return x, y

x, y = separate_cols(df)
train_x = x[:math.floor(0.7*len(df))]
train_y = y[:math.floor(0.7*len(df))]
test_x = x[math.floor(0.7*len(df)):]
test_y = y[math.floor(0.7*len(df)):]

len(df), len(train_x), len(train_y), len(test_x), len(test_y)

(241, 168, 168, 73, 73)

In [8]:
model = sklearn_crfsuite.CRF(
    algorithm = 'l2sgd', 
    c2=1,
    max_iterations=100, 
    all_possible_transitions=True,
    verbose=False
)

model.fit(train_x, train_y)
y_pred = model.predict(test_x)

In [9]:
labels = list(model.classes_)
labels.remove('O')

f1 = metrics.flat_f1_score(test_y, y_pred, 
                      average='weighted', labels=labels)
print(f1)

0.9665589021850224


E:\Instalations\envs\ner\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [10]:
print(metrics.flat_classification_report(
    test_y, y_pred, labels=labels, digits=3
))

                         precision    recall  f1-score   support

                 B-nome      1.000     0.963     0.981        27
                 I-nome      1.000     0.963     0.981        80
                B-cargo      0.963     1.000     0.981        26
                I-cargo      0.976     1.000     0.988       166
               B-classe      1.000     0.913     0.955        23
               I-classe      1.000     0.913     0.955        23
     B-fundamento_legal      0.986     0.973     0.979        73
     I-fundamento_legal      0.962     0.975     0.968       709
            B-matricula      1.000     0.963     0.981        27
            I-matricula      1.000     0.974     0.987        38
               B-padrao      1.000     0.957     0.978        23
               I-padrao      1.000     0.957     0.978        23
               B-quadro      1.000     1.000     1.000        23
               I-quadro      1.000     1.000     1.000       235
             B-processo 

E:\Instalations\envs\ner\lib\site-packages\sklearn\utils\validation.py:68: FutureWarning: Pass labels=['B-nome', 'I-nome', 'B-cargo', 'I-cargo', 'B-classe', 'I-classe', 'B-fundamento_legal', 'I-fundamento_legal', 'B-matricula', 'I-matricula', 'B-padrao', 'I-padrao', 'B-quadro', 'I-quadro', 'B-processo', 'I-processo', 'B-vigencia', 'I-vigencia', 'B-tipo_de_aposentadoria', 'I-tipo_de_aposentadoria', 'B-matricula_siape', 'I-matricula_siape'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
E:\Instalations\envs\ner\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Instalations\envs\ner\lib\site-packages\sklearn\metrics\_classification.py:1221:

In [11]:
# Save trained model
joblib.dump(model, "aposentadoria_ner.pkl")

['aposentadoria_ner.pkl']